# **Querying a NEO4J graph with LangChain**

In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
NEO4J_URI=userdata.get('NEO4J_URI')
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=userdata.get('NEO4J_PASSWORD')

In [4]:
#connecting to graph
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

<ipython-input-4-c5cf07494708>:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [5]:
graph

In [6]:
groq_api_key=userdata.get('GROQ_API')

In [7]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c7a0d484690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c7a0d4da850>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

Converting text into a Graph

In [8]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [9]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.8 MB/s eta 0:00:00


In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [11]:
#converting docs to graph
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [12]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Kimbal Musk', type='Person', properties={}), Node(id='Spacex', type='Organization', properties={}), Node(id='Tesla, Inc.', type='Organization', properties={}), Node(id='X Corp.', type='Organization', properties={}), Node(id='The Boring Company', type='Organization', properties={}), Node(id='Xai', type='Organization', properties={}), Node(id='Neuralink', type='Organization', properties={}), Node(id='Openai', type='Organization', properties={}), Node(id='University Of Pretoria', type='Educational institution', properties={}), Node(id="Queen'S University At Kingston", type='Educational institution', properties={}), Node(id='University Of Pennsylvania', type='Educational institution', properties={}), Node(id='Stanford University', type='Educational institution', properties={}), Node(id='Zip2', type='Orga

In [14]:
#Previewing the nodes
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal Musk', type='Person', properties={}),
 Node(id='Spacex', type='Organization', properties={}),
 Node(id='Tesla, Inc.', type='Organization', properties={}),
 Node(id='X Corp.', type='Organization', properties={}),
 Node(id='The Boring Company', type='Organization', properties={}),
 Node(id='Xai', type='Organization', properties={}),
 Node(id='Neuralink', type='Organization', properties={}),
 Node(id='Openai', type='Organization', properties={}),
 Node(id='University Of Pretoria', type='Educational institution', properties={}),
 Node(id="Queen'S University At Kingston", type='Educational institution', properties={}),
 Node(id='University Of Pennsylvania', type='Educational institution', properties={}),
 Node(id='Stanford University', type='Educational institution', properties={}),
 Node(id='Zip2', type='Organizati

In [15]:
#Looking through the extracted relationships
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Kimbal Musk', type='Person', properties={}), type='SIBLING', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Organization', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Organization', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Organization', properties={}),

Reading a csv file to add to the graph

In [16]:
#Adding a movie dataset to a graph
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.Released = date(row.Released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [17]:
#loading
graph.query(movie_query)

[]

In [18]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {name: STRING, born: INTEGER}
Movie {Released: INTEGER, title: STRING, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:acted_in]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


Querying the updated graph

In [20]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7c7a1bf7b610>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c7a0d484690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c7a0d4da850>, model_name='Gemma2-9b-I

In [21]:
#Actual NL to CQL
response=chain.invoke({"query":"Who was the director of the movie Jumanji"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Jumanji"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Joe Johnston'}]

> Finished chain.


{'query': 'Who was the director of the movie Jumanji',
 'result': "I don't know the answer. \n"}

In [24]:
response=chain.invoke({"query":"Who was the director of the movie GoldenEye"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of the movie GoldenEye',
 'result': 'Martin Campbell  \n'}

In [25]:
response=chain.invoke({"query":"Who was the director in movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title:"Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': 'Martin Scorsese  \n'}